In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [19]:
# 원본데이터 불러오기
origin_2023 = pd.read_csv('../1.Real_Estate_Market_Data/Actual_Transaction_Price/아파트(매매)_실거래가_2023.csv', encoding='CP949')
origin_2022 = pd.read_csv('../1.Real_Estate_Market_Data/Actual_Transaction_Price/아파트(매매)_실거래가_2022.csv', encoding='CP949')
origin_2021 = pd.read_csv('../1.Real_Estate_Market_Data/Actual_Transaction_Price/아파트(매매)_실거래가_2021.csv', encoding='CP949')
origin_2020 = pd.read_csv('../1.Real_Estate_Market_Data/Actual_Transaction_Price/아파트(매매)_실거래가_2020.csv', encoding='CP949')
origin_2019 = pd.read_csv('../1.Real_Estate_Market_Data/Actual_Transaction_Price/아파트(매매)_실거래가_2019.csv', encoding='CP949')

# 위도 경도 표시한 데이터 불러오기
landl_2023_1 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2023_1.csv', encoding="utf-8")
landl_2023_2 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2023_2.csv', encoding="utf-8")
landl_2022 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2022_완료.csv', encoding="CP949")
landl_2021 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2021_완료.csv', encoding="utf-8")
landl_2020_1 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2020_1_56989.csv', encoding="utf-8")
landl_2020_2 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2020_54651_82249.csv', encoding="utf-8")
landl_2020_3 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2020_82249_84135.csv', encoding="utf-8")
landl_2019_1 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_1_13129.csv', encoding='utf-8')
landl_2019_2 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_13130_18399.csv', encoding='utf-8')
landl_2019_3 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_18400_37544.csv', encoding='utf-8')
landl_2019_4 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_37545_37549.csv', encoding='utf-8')
landl_2019_5 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_37550_55129.csv', encoding='utf-8')
landl_2019_6 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_55130_68659.csv', encoding='utf-8')
landl_2019_7 = pd.read_csv('forDP/l_and_l/geocoded_addresses_2019_66549_75097.csv', encoding='utf-8')

# 브랜드 아파트 가져오기(1-28은 순위 29~ 가나다순정렬)
brand_apt = pd.read_csv('forDP/브랜드아파트순위.csv')

In [20]:
landl_2019_4

,no,address,latitude,longitude
0,37545,서울특별시 성북구 북악산로 813,37.601139,127.014946
1,37546,서울특별시 성동구 금호로 173,37.556279,127.018152
2,37547,서울특별시 성북구 아리랑로 89,37.600770,127.013617
3,37548,서울특별시 광진구 능동로 50,37.535675,127.069180
4,37549,서울특별시 강서구 마곡중앙1로 71,37.556908,126.817875


In [21]:
# 년도별로 위도경도 데이터 합치기, 중복 제거
geo_2023 = pd.concat([landl_2023_1, landl_2023_2], axis=0, ignore_index=True)
geo_2023 = geo_2023.drop_duplicates()

geo_2022 = landl_2022
geo_2022 = geo_2022.drop_duplicates()

geo_2021 = landl_2021
geo_2021 = geo_2021.drop_duplicates()

geo_2020 = pd.concat([landl_2020_1, landl_2020_2, landl_2020_3], ignore_index=True)
geo_2020 = geo_2020.drop_duplicates()

geo_2019 = pd.concat([landl_2019_1, landl_2019_2, landl_2019_3, landl_2019_4, landl_2019_5, landl_2019_6, landl_2019_7], ignore_index=True)
geo_2019 = geo_2019.drop_duplicates()

In [22]:
# 인덱스용 컬럼 타입 및 값 변경
def process_geo_dataframe(df, year):
    # 컬럼명 'no'를 'NO'로 변경
    df = df.rename(columns={'no': 'NO'})
    
    # 'NO' 컬럼을 int로 변환
    df['NO'] = df['NO'].astype('int')
    
    # 'NO' 컬럼을 년도와 5자리 zero-padded 번호로 생성
    df['NO'] = f'{year}_' + df['NO'].astype('str').str.zfill(5)
    
    # 필요한 컬럼만 선택
    df = df[['NO', 'latitude', 'longitude', 'address']]
    
    return df

def process_origin_dataframe(df, year):
    # 'NO' 컬럼을 int로 변환
    df['NO'] = df['NO'].astype('int')
    
    # 'NO' 컬럼을 년도와 5자리 zero-padded 번호로 생성
    df['NO'] = f'{year}_' + df['NO'].astype('str').str.zfill(5)
    
    # '거래금액(만원)'에서 쉼표 제거 후 정수형으로 변환
    df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',', '').astype(int)
        
    return df

In [23]:
# Geo 데이터프레임 병합용 리스트
years = [2019, 2020, 2021, 2022, 2023]

geo_list = []
for year, geo_df in zip(years, [geo_2019, geo_2020, geo_2021, geo_2022, geo_2023]):
    processed_geo = process_geo_dataframe(geo_df, year)
    geo_list.append(processed_geo)  # 리스트에 추가
    processed_geo.to_csv(f'Data_Preprocessing/processed_geo_{year}.csv', index=False)

# 모든 Geo 데이터프레임 병합
combined_geo = pd.concat(geo_list, ignore_index=True)

# Origin 데이터프레임 병합용 리스트
origin_list = []
for year, origin_df in zip(years, [origin_2019, origin_2020, origin_2021, origin_2022, origin_2023]):
    processed_origin = process_origin_dataframe(origin_df, year)
    origin_list.append(processed_origin)  # 리스트에 추가
    processed_origin.to_csv(f'Data_Preprocessing/processed_origin_{year}.csv', index=False)

# 모든 Origin 데이터프레임 병합
combined_origin = pd.concat(origin_list, ignore_index=True)

# 확인
print("Combined Geo DataFrame:")
print(combined_geo.head())

print("Combined Origin DataFrame:")
print(combined_origin.head())

# 병합된 데이터프레임을 CSV로 저장
combined_geo.to_csv("Data_Preprocessing/combined_geo.csv", index=False)
combined_origin.to_csv("Data_Preprocessing/combined_origin.csv", index=False)

Combined Geo DataFrame:
           NO   latitude   longitude               address
0  2019_00001  37.569628  127.041524     서울특별시 성동구 살곶이길 50
1  2019_00002  37.589283  127.072500     서울특별시 중랑구 중랑천로 43
2  2019_00003  37.574609  127.060129  서울특별시 동대문구 전농로10길 20
3  2019_00004  37.616686  127.001842    서울특별시 성북구 보국문로 167
4  2019_00005  37.576338  127.074573  서울특별시 동대문구 장안벚꽃로1길 7
Combined Origin DataFrame:
           NO              시군구     번지    본번  부번         단지명  전용면적(㎡)    계약년월  \
0  2019_00001    서울특별시 성동구 마장동    818   818   0          현대  134.790  201912   
1  2019_00002    서울특별시 중랑구 면목동   1519  1519   0          신성   84.980  201912   
2  2019_00003  서울특별시 동대문구 답십리동     80    80   0     답십리청솔우성   59.740  201912   
3  2019_00004    서울특별시 성북구 정릉동    780   780   0  산장아파트가동,나동   71.500  201912   
4  2019_00005   서울특별시 동대문구 장안동  329-2   329   2     장안힐스테이트   59.928  201912   

   계약일  거래금액(만원)  동   층 매수자 매도자  건축년도        도로명 해제사유발생일 거래유형 중개사소재지 등기일자  
0   31     88000  -  17   -   -  1998 

In [24]:
def analyze_missing_values(geo_dfs, origin_dfs, years):
    # 먼저 모든 미싱 밸류 수 출력
    print("Missing Values Summary:")
    print("\nGeo DataFrames:")
    for year, geo_df in zip(years, geo_dfs):
        print(f"\nYear: {year}")
        geo_missing = geo_df.isnull().sum()[geo_df.isnull().sum() > 0]
        print(geo_missing)
    
    print("\nOrigin DataFrames:")
    for year, origin_df in zip(years, origin_dfs):
        print(f"\nYear: {year}")
        origin_missing = origin_df.isnull().sum()[origin_df.isnull().sum() > 0]
        print(origin_missing)
    
    # 그 다음 각 년도별 결측치 행 출력 및 저장
    print("\n\nMissing Value Rows:")
    for year, geo_df, origin_df in zip(years, geo_dfs, origin_dfs):
        print(f"\nYear: {year}")
        
        # Geo DataFrame 결측치 행
        print("\nGeo DataFrame - Rows with missing values:")
        geo_mv = geo_df[geo_df.isnull().any(axis=1)]
        print(geo_mv.to_string() if not geo_mv.empty else "No missing values")
        
        # Origin DataFrame 결측치 행
        print("\nOrigin DataFrame - Rows with missing values:")
        origin_mv = origin_df[origin_df.isnull().any(axis=1)]
        print(origin_mv.to_string() if not origin_mv.empty else "No missing values")
        
        # CSV로 저장
        geo_mv.to_csv(f'Data_Preprocessing/missing_values_geo_{year}.csv', index=False)
        origin_mv.to_csv(f'Data_Preprocessing/missing_values_origin_{year}.csv', index=False)

In [25]:
years = [2023, 2022, 2021, 2020, 2019]
geo_dfs = [geo_2023, geo_2022, geo_2021, geo_2020, geo_2019]
origin_dfs = [origin_2023, origin_2022, origin_2021, origin_2020, origin_2019]

analyze_missing_values(geo_dfs, origin_dfs, years)

Missing Values Summary:

Geo DataFrames:

Year: 2023
latitude     73
longitude    73
dtype: int64

Year: 2022
latitude     145
longitude    145
dtype: int64

Year: 2021
latitude     454
longitude    454
dtype: int64

Year: 2020
latitude     739
longitude    739
dtype: int64

Year: 2019
latitude     572
longitude    572
dtype: int64

Origin DataFrames:

Year: 2023
Series([], dtype: int64)

Year: 2022
Series([], dtype: int64)

Year: 2021
Series([], dtype: int64)

Year: 2020
Series([], dtype: int64)

Year: 2019
Series([], dtype: int64)


Missing Value Rows:

Year: 2023

Geo DataFrame - Rows with missing values:
          no                 address  latitude  longitude
1057    1058      서울특별시 은평구 연서로19길 9       NaN        NaN
1086    1087      서울특별시 은평구 연서로19길 9       NaN        NaN
2379    2380    서울특별시 강남구 남부순환로 2803       NaN        NaN
2805    2806    서울특별시 성동구 왕십리로 66-15       NaN        NaN
3121    3122    서울특별시 강남구 남부순환로 2803       NaN        NaN
3971    3972      서울특별시 은평구 연서로19길 9

In [26]:
combined_geo.info(), combined_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   NO         251096 non-null  object 
 1   latitude   249113 non-null  float64
 2   longitude  249113 non-null  float64
 3   address    251096 non-null  object 
dtypes: float64(2), object(2)
memory usage: 7.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   NO        251096 non-null  object 
 1   시군구       251096 non-null  object 
 2   번지        251096 non-null  object 
 3   본번        251096 non-null  int64  
 4   부번        251096 non-null  int64  
 5   단지명       251096 non-null  object 
 6   전용면적(㎡)   251096 non-null  float64
 7   계약년월      251096 non-null  int64  
 8   계약일       251096 non-null  int64  
 9   거래금액(만원)  251096 non-null  int32  
 

(None, None)

In [27]:
merged_df = pd.merge(combined_geo, combined_origin, how='inner', on='NO')
merged_df[['계약년월', '계약일']] = merged_df[['계약년월', '계약일']].astype('str')
merged_df["계약일자"] = pd.to_datetime(merged_df['계약년월'] + merged_df['계약일'])
merged_df = merged_df[['NO', '계약일자', '단지명', '전용면적(㎡)', '거래금액(만원)', 
                        '층', '건축년도', 'latitude', 'longitude']]
merged_df.columns = ['NO', 'contract_date', 'apt_name', 'exclusive_area_m2', 
                        'price_man_won', 'floor', 'construction_year', 'latitude', 'longitude']
merged_df.info()
merged_df.head(), merged_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   NO                 251096 non-null  object        
 1   contract_date      251096 non-null  datetime64[ns]
 2   apt_name           251096 non-null  object        
 3   exclusive_area_m2  251096 non-null  float64       
 4   price_man_won      251096 non-null  int32         
 5   floor              251096 non-null  int64         
 6   construction_year  251096 non-null  int64         
 7   latitude           249113 non-null  float64       
 8   longitude          249113 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(2)
memory usage: 16.3+ MB


(           NO contract_date    apt_name  exclusive_area_m2  price_man_won  \
 0  2019_00001    2019-12-31          현대            134.790          88000   
 1  2019_00002    2019-12-31          신성             84.980          47800   
 2  2019_00003    2019-12-31     답십리청솔우성             59.740          54500   
 3  2019_00004    2019-12-31  산장아파트가동,나동             71.500          32200   
 4  2019_00005    2019-12-31     장안힐스테이트             59.928          65500   
 
    floor  construction_year   latitude   longitude  
 0     17               1998  37.569628  127.041524  
 1      9               1998  37.589283  127.072500  
 2     21               2000  37.574609  127.060129  
 3      2               1977  37.616686  127.001842  
 4     12               2007  37.576338  127.074573  ,
                 NO contract_date  apt_name  exclusive_area_m2  price_man_won  \
 251091  2023_35638    2023-01-02      메트하임              14.03          17000   
 251092  2023_35639    2023-01-02  대경아르체아파트

In [28]:
# CPI 데이터 및 조정 함수 정의
cpi_data = {
    "year": [2019, 2020, 2021, 2022, 2023],
    "cpi": [99.5, 100, 107.7, 111.6, 111.6],  # 2020년을 기준
}

cpi_df = pd.DataFrame(cpi_data)

# CPI 조정 함수
def adjust_for_cpi(price, contract_date, target_year_cpi=111.6):
    year = contract_date.year
    year_cpi = cpi_df.loc[cpi_df['year'] == year, 'cpi'].values[0]
    return price * (target_year_cpi / year_cpi)

# 실질 가격 계산
merged_df['adjusted_price'] = merged_df.apply(
    lambda row: adjust_for_cpi(row['price_man_won'], row['contract_date']), axis=1
)

# 결과 확인
print(merged_df[['contract_date', 'price_man_won', 'adjusted_price']].head())


  contract_date  price_man_won  adjusted_price
0    2019-12-31          88000    98701.507538
1    2019-12-31          47800    53612.864322
2    2019-12-31          54500    61127.638191
3    2019-12-31          32200    36115.778894
4    2019-12-31          65500    73465.326633


In [29]:
merged_df.to_csv('Data_Preprocessing/apt_information_with_l_and_l.csv', index=False, encoding='utf-8')

In [30]:
brand_apt.columns = ['rank', 'brand']

In [31]:
# 'yes_brand'와 'no_brand' 컬럼 생성
# brand_apt의 'brand' 값을 리스트로 추출
brand_list = brand_apt['brand'].tolist()

# 'apt_name'이 brand_list의 값 중 하나를 포함하는지 확인
def check_brand(apt_name):
    if any(brand in apt_name for brand in brand_list):  # apt_name이 brand_list에 포함된 경우
        return 1, 0  # yes_brand=1, no_brand=0
    else:  # 포함되지 않은 경우
        return 0, 1  # yes_brand=0, no_brand=1

# 'yes_brand'와 'no_brand' 컬럼 생성
merged_df[['yes_brand', 'no_brand']] = merged_df['apt_name'].apply(lambda x: pd.Series(check_brand(x)))


In [ ]:
merged_df = merged_df[['NO', 'contract_date', 'exclusive_area_m2', 'floor',
                        'construction_year', 'latitude', 'longitude', 
                        'yes_brand', 'no_brand', 'price_man_won', 'adjusted_price']]
merged_df.info()
# NO와 계약일자를 제외한 컬럼 모두 숫자형(정수형+실수형)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   NO                 251096 non-null  object        
 1   contract_date      251096 non-null  datetime64[ns]
 2   exclusive_area_m2  251096 non-null  float64       
 3   floor              251096 non-null  int64         
 4   construction_year  251096 non-null  int64         
 5   latitude           249113 non-null  float64       
 6   longitude          249113 non-null  float64       
 7   yes_brand          251096 non-null  int64         
 8   no_brand           251096 non-null  int64         
 9   price_man_won      251096 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(1), int64(4), object(1)
memory usage: 18.2+ MB


In [17]:
merged_df.to_csv('Data_Preprocessing/apt_information_brand_l_and_l.csv', index=False, encoding='utf-8')